# Milestone 3 — Data Cleaning

**Dataset:** clinic_appointments_en.csv  
**Goal:** Inspect and clean the data so it’s analysis-ready.  
This notebook:
1) Audits missing values, duplicates, and column formats  
2) Parses/standardizes columns (dates, categorical values, ints)  
3) Checks/handles outliers using IQR “winsorizing” (capping)  
4) Applies simple validity rules (e.g., non-negative distances)  
5) Saves a cleaned file for the next milestone


In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

In [2]:
df_raw = pd.read_csv("clinic_appointments_en.csv")
df = df_raw.copy()
print(df.shape)
df.head()

(5400, 10)


,date,age,gender,weekend,prior_no_shows,distance_km,lead_time_days,sms_reminder,rain,no_show
0,2025-03-01,75,Female,1,0,3.40,6,0,0,1
1,2025-03-01,30,Male,1,0,4.39,5,0,0,1
2,2025-03-01,46,Female,1,1,3.81,6,0,0,0
3,2025-03-01,57,Female,1,0,0.32,6,0,1,0
4,2025-03-01,57,Male,1,0,0.78,1,1,0,0


## Data audit (what needs cleaning?)
We’ll check:
- missing values
- duplicates
- data types
- value ranges for key columns

In [3]:
display(df.info())
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5400 entries, 0 to 5399
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            5400 non-null   object 
 1   age             5400 non-null   int64  
 2   gender          5400 non-null   object 
 3   weekend         5400 non-null   int64  
 4   prior_no_shows  5400 non-null   int64  
 5   distance_km     5400 non-null   float64
 6   lead_time_days  5400 non-null   int64  
 7   sms_reminder    5400 non-null   int64  
 8   rain            5400 non-null   int64  
 9   no_show         5400 non-null   int64  
dtypes: float64(1), int64(7), object(2)
memory usage: 422.0+ KB


None

date              0
age               0
gender            0
weekend           0
prior_no_shows    0
distance_km       0
lead_time_days    0
sms_reminder      0
rain              0
no_show           0
dtype: int64

In [4]:
dup_count = df.duplicated().sum()
print("Duplicate rows:", dup_count)

Duplicate rows: 0


In [5]:
audit_cols = ["age","prior_no_shows","distance_km","lead_time_days","sms_reminder","rain","weekend","no_show","gender"]
for c in audit_cols:
    if c in df.columns:
        print(f"\n=== {c} ===")
        if df[c].dtype == "object":
            print(df[c].value_counts().head(10))
        else:
            print(df[c].describe())


=== age ===
count    5400.000000
mean       44.163889
std        17.724006
min         0.000000
25%        32.000000
50%        44.000000
75%        56.000000
max        95.000000
Name: age, dtype: float64

=== prior_no_shows ===
count    5400.000000
mean        0.397407
std         0.630632
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         4.000000
Name: prior_no_shows, dtype: float64

=== distance_km ===
count    5400.000000
mean        5.996044
std         4.239254
min         0.100000
25%         2.897500
50%         5.055000
75%         8.000000
max        32.070000
Name: distance_km, dtype: float64

=== lead_time_days ===
count    5400.000000
mean        6.702593
std         4.595962
min         0.000000
25%         3.000000
50%         7.000000
75%        10.000000
max        27.000000
Name: lead_time_days, dtype: float64

=== sms_reminder ===
count    5400.000000
mean        0.539074
std         0.498517
min         0.000000
25%   

## Standardize columns
- Parse date to datetime  
- Tidy gender values  
- Ensure binary flags are integers in {0,1}

In [6]:
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    bad_dates = df["date"].isna().sum()
    print("Unparseable dates:", bad_dates)

# 3b) Gender standardization (common variants -> Male/Female/Other)
if "gender" in df.columns:
    gmap = {
        "m": "Male", "male": "Male", "MALE": "Male",
        "f": "Female", "female": "Female", "FEMALE": "Female"
    }
    df["gender"] = (
        df["gender"]
        .astype(str)
        .str.strip()
        .str.title()
        .replace(gmap)
    )
    # Anything not Male/Female becomes Other (optional)
    df.loc[~df["gender"].isin(["Male","Female"]), "gender"] = "Other"

# 3c) Force integer types for binary columns
binary_cols = [c for c in ["weekend","sms_reminder","rain","no_show"] if c in df.columns]
for c in binary_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int).clip(0,1)

# Prior no-shows, lead_time_days should be non-negative integers
for c in ["prior_no_shows","lead_time_days"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).clip(lower=0).round().astype(int)

# Distance and age cleanups
if "distance_km" in df.columns:
    df["distance_km"] = pd.to_numeric(df["distance_km"], errors="coerce").fillna(0).clip(lower=0)

if "age" in df.columns:
    # Keep within human range 0–100 (adjust if your source allows older)
    df["age"] = pd.to_numeric(df["age"], errors="coerce").fillna(0).clip(0, 100).round().astype(int)


Unparseable dates: 0


## Outlier handling (IQR winsorizing)
For numeric columns, we cap extreme values outside **Q1 − 1.5·IQR** and **Q3 + 1.5·IQR**.  
This preserves rows but prevents a few extremes from dominating later plots.

In [7]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

def iqr_clip(s: pd.Series):
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return s.clip(lower, upper)

before_stats = df[numeric_cols].describe()

for c in numeric_cols:
    df[c] = iqr_clip(df[c])

after_stats = df[numeric_cols].describe()

print("Clipped columns:", numeric_cols)
display(before_stats.loc[["min","25%","50%","75%","max"]])
display(after_stats.loc[["min","25%","50%","75%","max"]])

Clipped columns: ['age', 'weekend', 'prior_no_shows', 'distance_km', 'lead_time_days', 'sms_reminder', 'rain', 'no_show']


,age,weekend,prior_no_shows,distance_km,lead_time_days,sms_reminder,rain,no_show
min,0.0,0.0,0.0,0.1000,0.0,0.0,0.0,0.0
25%,32.0,0.0,0.0,2.8975,3.0,0.0,0.0,0.0
50%,44.0,0.0,0.0,5.0550,7.0,1.0,0.0,0.0
75%,56.0,1.0,1.0,8.0000,10.0,1.0,1.0,0.0
max,95.0,1.0,4.0,32.0700,27.0,1.0,1.0,1.0


,age,weekend,prior_no_shows,distance_km,lead_time_days,sms_reminder,rain,no_show
min,0.0,0.0,0.0,0.10000,0.0,0.0,0.0,0.0
25%,32.0,0.0,0.0,2.89750,3.0,0.0,0.0,0.0
50%,44.0,0.0,0.0,5.05500,7.0,1.0,0.0,0.0
75%,56.0,1.0,1.0,8.00000,10.0,1.0,1.0,0.0
max,92.0,1.0,2.5,15.65375,20.5,1.0,1.0,0.0


## Validity checks
Make sure constraints still hold after cleaning.

In [9]:
checks = {}

if "age" in df.columns:
    checks["age_nonnegative"] = (df["age"] >= 0).all()
    checks["age_max_le_100"] = (df["age"] <= 100).all()

for c in ["lead_time_days","prior_no_shows","distance_km"]:
    if c in df.columns:
        checks[f"{c}_nonnegative"] = (df[c] >= 0).all()

for c in ["weekend","sms_reminder","rain","no_show"]:
    if c in df.columns:
        checks[f"{c}_binary"] = set(df[c].unique()).issubset({0,1})

checks


{'age_nonnegative': np.True_,
 'age_max_le_100': np.True_,
 'lead_time_days_nonnegative': np.True_,
 'prior_no_shows_nonnegative': np.True_,
 'distance_km_nonnegative': np.True_,
 'weekend_binary': True,
 'sms_reminder_binary': True,
 'rain_binary': True,
 'no_show_binary': True}

## Duplicates removal

In [10]:
dup_rows = df.duplicated()
print("Duplicates found:", dup_rows.sum())

Duplicates found: 0


## Final review

In [12]:
display(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5400 entries, 0 to 5399
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            5400 non-null   datetime64[ns]
 1   age             5400 non-null   int64         
 2   gender          5400 non-null   object        
 3   weekend         5400 non-null   int64         
 4   prior_no_shows  5400 non-null   float64       
 5   distance_km     5400 non-null   float64       
 6   lead_time_days  5400 non-null   float64       
 7   sms_reminder    5400 non-null   int64         
 8   rain            5400 non-null   int64         
 9   no_show         5400 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(5), object(1)
memory usage: 422.0+ KB


None

,date,age,gender,weekend,prior_no_shows,distance_km,lead_time_days,sms_reminder,rain,no_show
0,2025-03-01,75,Female,1,0.0,3.40,6.0,0,0,0
1,2025-03-01,30,Male,1,0.0,4.39,5.0,0,0,0
2,2025-03-01,46,Female,1,1.0,3.81,6.0,0,0,0
3,2025-03-01,57,Female,1,0.0,0.32,6.0,0,1,0
4,2025-03-01,57,Male,1,0.0,0.78,1.0,1,0,0
